In [1]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'1.3.1'

In [2]:
from ampligraph.datasets import load_from_rdf

In [5]:
%%time
X = load_from_rdf('.', 'sample_Game_KG.nt', rdf_format='nt', data_home='.')

CPU times: user 833 ms, sys: 90 ms, total: 923 ms
Wall time: 924 ms


In [6]:
X[:3, ]

array([['http://inf558.org/games#mig_47339', 'http://schema.org/url',
        'https://www.igdb.com/games/break-chance-memento'],
       ['N310c9edf320d492495dfbbfc1ca80cad',
        'http://inf558.org/games#memory_MB', '512'],
       ['http://inf558.org/games#mig_53044',
        'http://inf558.org/games#ratingValue', '-1']], dtype=object)

In [7]:
X.shape

(10000, 3)

In [10]:
from ampligraph.evaluation import train_test_split_no_unseen 

test_size1 = int(0.1 * X.shape[0])
test_size2 = int(0.05 * X.shape[0])

In [11]:
X_train, X_test = train_test_split_no_unseen(X, test_size=test_size2)

In [12]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (9500, 3)
Test set size:  (500, 3)


In [13]:
from ampligraph.latent_features import ComplEx

In [17]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=10, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [18]:
positives_filter = X

In [19]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)


Average Loss:   0.083291: 100%|██████████| 10/10 [00:56<00:00,  5.68s/epoch]


In [20]:
from ampligraph.evaluation import evaluate_performance

In [21]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.



 61%|██████▏   | 307/500 [00:25<00:15, 12.80it/s]


100%|██████████| 500/500 [00:41<00:00, 12.09it/s]


In [22]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.03
Hits@10: 0.06
Hits@3: 0.04
Hits@1: 0.02


In [23]:
model.get_embeddings(['http://inf558.org/games#mig_47339'], embedding_type='entity')

array([[ 0.07543745,  0.07814845,  0.13340354,  0.07674506, -0.095493  ,
         0.12731843, -0.02968431,  0.06432015, -0.1000028 , -0.04162085,
         0.01949068,  0.03616851, -0.15003943, -0.03254428,  0.07616   ,
        -0.02537142, -0.03702584,  0.03467752,  0.11109807,  0.13050926,
         0.13613036, -0.01466509,  0.03947484, -0.12905993, -0.12003832,
        -0.13657439,  0.08668496,  0.12139896,  0.07323246, -0.10078449,
         0.08327837, -0.1019605 , -0.11969604, -0.08740832,  0.08489975,
        -0.13252318, -0.11481824, -0.12277126, -0.05196132, -0.12820666,
        -0.0561395 , -0.13401665,  0.08442388,  0.13905619,  0.01710225,
        -0.09045249, -0.00594154, -0.06928227,  0.0870918 , -0.10656806,
         0.0934609 ,  0.05162122,  0.11196775, -0.07661655,  0.03561096,
         0.1101543 , -0.10954233,  0.02615547,  0.10349514,  0.03210045,
         0.02246342, -0.03234481,  0.12004673,  0.10158699, -0.02128778,
         0.10461398,  0.14635967, -0.0863873 ,  0.0

In [25]:
from ampligraph.utils import save_model, restore_model

In [26]:
save_model(model, model_name_path = "game_kge_model.pkl")

In [27]:
restored_model = restore_model(model_name_path = "game_kge_model.pkl")

In [28]:
restored_model.get_embeddings(['http://inf558.org/games#mig_47339'], embedding_type='entity')

array([[ 0.07543745,  0.07814845,  0.13340354,  0.07674506, -0.095493  ,
         0.12731843, -0.02968431,  0.06432015, -0.1000028 , -0.04162085,
         0.01949068,  0.03616851, -0.15003943, -0.03254428,  0.07616   ,
        -0.02537142, -0.03702584,  0.03467752,  0.11109807,  0.13050926,
         0.13613036, -0.01466509,  0.03947484, -0.12905993, -0.12003832,
        -0.13657439,  0.08668496,  0.12139896,  0.07323246, -0.10078449,
         0.08327837, -0.1019605 , -0.11969604, -0.08740832,  0.08489975,
        -0.13252318, -0.11481824, -0.12277126, -0.05196132, -0.12820666,
        -0.0561395 , -0.13401665,  0.08442388,  0.13905619,  0.01710225,
        -0.09045249, -0.00594154, -0.06928227,  0.0870918 , -0.10656806,
         0.0934609 ,  0.05162122,  0.11196775, -0.07661655,  0.03561096,
         0.1101543 , -0.10954233,  0.02615547,  0.10349514,  0.03210045,
         0.02246342, -0.03234481,  0.12004673,  0.10158699, -0.02128778,
         0.10461398,  0.14635967, -0.0863873 ,  0.0